In [34]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/kdt_240424/m5_머신러닝/dataset/train.csv')

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.Name.value_counts()

Name
Braund, Mr. Owen Harris                     1
Boulos, Mr. Hanna                           1
Frolicher-Stehli, Mr. Maxmillian            1
Gilinski, Mr. Eliezer                       1
Murdlin, Mr. Joseph                         1
                                           ..
Kelly, Miss. Anna Katherine "Annie Kate"    1
McCoy, Mr. Bernard                          1
Johnson, Mr. William Cahoone Jr             1
Keane, Miss. Nora A                         1
Dooley, Mr. Patrick                         1
Name: count, Length: 891, dtype: int64

In [35]:
df['Title'] = df['Name'].apply(lambda x: re.findall(r'\b([a-zA-Z]+)\.', x)[0])

In [37]:
df.Title.value_counts()

Title
Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Mlle          2
Major         2
Col           2
Countess      1
Capt          1
Ms            1
Sir           1
Lady          1
Mme           1
Don           1
Jonkheer      1
Name: count, dtype: int64

In [39]:
pd.crosstab(df["Title"], df["Sex"])

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


In [44]:
print(pd.crosstab(df["Title"], df["Sex"]))

Sex         female  male
Title                   
FemaleRare       6     0
MaleRare         0    14
Master           0    40
Miss           182     0
Mr               0   517
Mrs            125     0
Rare             1     6


In [40]:
df['Title'] = df['Title'].replace(["Capt", "Col", "Don", "Jonkheer", "Major", "Rev", "Sir"], "MaleRare")
df['Title'] = df['Title'].replace(["Countess", "Lady", "Mlle", "Mme", "Ms"], "FemaleRare")
df['Title'] = df['Title'].replace(["Dr"], "Rare")

In [41]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [43]:
df.Title.value_counts()

Title
Mr            517
Miss          182
Mrs           125
Master         40
MaleRare       14
Rare            7
FemaleRare      6
Name: count, dtype: int64

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/kdt_240424/m5_머신러닝/dataset/train.csv')


# 전처리 함수
def transform_features(df):

    df_numeric = df.select_dtypes(include='number')
    print(df_numeric.corr()['Survived'])

    df['Title'] = df['Name'].apply(lambda x: re.findall(r'\b([a-zA-Z]+)\.', x)[0])
    print(pd.crosstab(df["Title"], df["Sex"]))

    # 필요없는 열 제거
    df.drop(columns = ['PassengerId','Name','Ticket','Cabin','Fare'], inplace=True)


    df['Title'] = df['Title'].replace(["Mr","Master","Capt", "Col", "Don", "Jonkheer", "Major", "Rev", "Sir"], "MaleRare")
    df['Title'] = df['Title'].replace(["Miss","Mrs","Countess", "Lady", "Mlle", "Mme", "Ms"], "FemaleRare")
    df['Title'] = df['Title'].replace(["Dr"], "Rare")

    # 결측치 제거
    df['Embarked'].fillna(df['Embarked'].mode()[0], inplace = True)
    df.Age.fillna(df.Age.mean(), inplace=True)
    print(df.isnull().sum())


    # age_group을 6개 범주로 확대
    bins = [0,5,12,18,35,60,100]
    labels = ['Infant','Child','Teenager','Yound Adult','Adult','Senior']
    df['age_group'] = pd.cut(df['Age'], bins=bins, labels = labels)
    df.drop(columns=['Age'], inplace = True)



    category_column = ['age_group','Sex','Pclass','Embarked','SibSp','Parch','Title']
    for column in category_column:
        le = LabelEncoder()
        le = le.fit(df[column])
        df[column] = le.transform(df[column])
    return df

In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from sklearn.preprocessing import StandardScaler,OneHotEncoder, LabelEncoder
import re
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('/content/drive/MyDrive/kdt_240424/m5_머신러닝/dataset/train.csv')


# 전처리 함수
def transform_features(df):

    df_numeric = df.select_dtypes(include='number')
    print(df_numeric.corr()['Survived'])

    df['Title'] = df['Name'].apply(lambda x: re.findall(r'\b([a-zA-Z]+)\.', x)[0])
    print(pd.crosstab(df["Title"], df["Sex"]))

    # 필요없는 열 제거
    df.drop(columns = ['PassengerId','Name','Ticket','Cabin'], inplace=True)


    df['Title'] = df['Title'].replace(["Mr","Master","Capt", "Col", "Don", "Jonkheer", "Major", "Rev", "Sir"], "MaleRare")
    df['Title'] = df['Title'].replace(["Miss","Mrs","Countess", "Lady", "Mlle", "Mme", "Ms"], "FemaleRare")
    df['Title'] = df['Title'].replace(["Dr"], "Rare")

    # 결측치 제거
    df['Embarked'].fillna(df['Embarked'].mode()[0], inplace = True)
    df['Age'].fillna(df.Age.mean(), inplace=True)
    df['Fare'].fillna(0,inplace=True)
    print(df.isnull().sum())


    # 파생변수 생성
    # df['family_size'] = df['SibSp']+df['Parch']+1
    # 필요없는 열 제거
    # df.drop(columns=['SibSp','Parch'],inplace=True)


    # age_group을 6개 범주로 확대
    bins = [0,5,12,18,35,60,100]
    labels = ['Infant','Child','Teenager','Yound Adult','Adult','Senior']
    df['age_group'] = pd.cut(df['Age'], bins=bins, labels = labels)
    df.drop(columns=['Age'], inplace = True)


    # category_column = ['age_group','Sex','Pclass','Embarked']
    # for column in category_column:
    #     df= pd.get_dummies(df,columns=[column])



    category_column = ['age_group','Sex','Pclass','Embarked','SibSp','Parch','Title']
    for column in category_column:
        le = LabelEncoder()
        le = le.fit(df[column])
        df[column] = le.transform(df[column])
    return df

# 분류 평가 함수
def evaluate(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:,1]

    cla_report = classification_report(y_test,y_pred)

    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred)
    recall = recall_score(y_test,y_pred)
    f1 = f1_score(y_test,y_pred)
    roc_auc = roc_auc_score(y_test,y_proba)

    print(f"Classification Report:\n {cla_report}")
    print(f'정확도 : {accuracy:.4f}')
    print(f"정밀도 :  {precision:.4f}")
    print(f"재현율 : {recall:.4f}")
    print(f"f1 스코어 : {f1:.4f}")
    print(f"Roc Auc : {roc_auc:.4f}")
    print()

# 전처리 함수를 통해 전처리 한 df 반환
df = transform_features(df)
print("전처리 완료")

print("="*80)

# 변수 설정
X = df.drop('Survived', axis=1)
y = df['Survived']


# 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3, random_state = 42)

# 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 모델링 및 평가

decision_tree_clf = DecisionTreeClassifier()
print(f"Decision Tree의 평가")
evaluate(decision_tree_clf, X_train, X_test, y_train, y_test)

random_forest_clf = RandomForestClassifier()
print(f"Random Forest의 평가")
evaluate(random_forest_clf, X_train, X_test, y_train, y_test)

logistic_regression_clf = LogisticRegression()
print(f"Logistic Regression의 평가")
evaluate(logistic_regression_clf, X_train, X_test, y_train, y_test)


print("평가 완료")
print("="*80)

# 파라미터 생성
param_dt = {
    'criterion': ['gini', 'entropy'],  # 분할 기준: 'gini' (지니 불순도), 'entropy' (엔트로피)
    'max_depth': [None, 10, 20, 30, 50],  # 트리의 최대 깊이
    'min_samples_split': [2, 5, 10],  # 노드를 분할하기 위한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4]  # 리프 노드에 필요한 최소 샘플 수
}

param_lr = {
    'penalty': ['l1', 'l2'],  # 규제 유형: 'l1' (Lasso), 'l2' (Ridge)
    'C': [0.1, 1, 10, 50, 100],  # 규제 강도의 역수
    'solver': ['liblinear', 'saga']  # 최적화에 사용할 알고리즘
}
param_rf = {
    'n_estimators' : [50, 100, 200],
    'max_depth': [None, 10, 20],
    'max_features' : ['auto', 'sqrt', 'log2'],
    'min_samples_split' : [2, 5, 10],
    'min_samples_leaf' : [1, 2, 4]
    'max_depth':[2,3,5,10,12],
    'min_samples_split':[2,3,5],
    'min_samples_leaf':[1,5,8,10]
}


# Decision Tree 최적화
grid_search_dt = GridSearchCV(estimator=decision_tree_clf, param_grid=param_dt, cv=5, n_jobs=-1, verbose=0)
grid_search_dt.fit(X_train, y_train)
best_dt = grid_search_dt.best_estimator_
print(grid_search_dt.best_params_)
print(f"최적 파라미터 Decision Tree의 평가")
evaluate(best_dt, X_train, X_test, y_train, y_test)

# Random Forest 최적화
grid_search_rf = GridSearchCV(estimator=random_forest_clf, param_grid=param_rf, cv=5, n_jobs=-1, verbose=0)
grid_search_rf.fit(X_train, y_train)
best_rf = grid_search_rf.best_estimator_
print(grid_search_rf.best_params_)
print(f"최적 파라미터 Random Forest의 평가")
evaluate(best_rf, X_train, X_test, y_train, y_test)

# Logistic Regression 최적화
grid_search_lr = GridSearchCV(estimator=logistic_regression_clf, param_grid=param_lr, cv=5, n_jobs=-1, verbose=0)
grid_search_lr.fit(X_train, y_train)
best_lr = grid_search_lr.best_estimator_
print(grid_search_lr.best_params_)
print(f"최적 파라미터 Logistic Regression의 평가")
evaluate(best_lr, X_train, X_test, y_train, y_test)

print("최적화 평가 완료")
print("=" * 80)

PassengerId   -0.005007
Survived       1.000000
Pclass        -0.338481
Age           -0.077221
SibSp         -0.035322
Parch          0.081629
Fare           0.257307
Name: Survived, dtype: float64
Sex       female  male
Title                 
Capt           0     1
Col            0     2
Countess       1     0
Don            0     1
Dr             1     6
Jonkheer       0     1
Lady           1     0
Major          0     2
Master         0    40
Miss         182     0
Mlle           2     0
Mme            1     0
Mr             0   517
Mrs          125     0
Ms             1     0
Rev            0     6
Sir            0     1
Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Embarked    0
Title       0
dtype: int64
전처리 완료
Decision Tree의 평가
Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.92      0.85       157
           1       0.85      0.66      0.74       111

    accuracy              

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0,inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
    return df

# 레이블 인코딩 수행.
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 Data Preprocessing 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt

# 사용자 정의 함수
def train_and_evaluate(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)
    confusion = confusion_matrix(y_test, y_pred)

    print(f'오차 행렬:\n{confusion}')
    print(f'정확도: {accuracy:.4f}')
    print(f'정밀도: {precision:.4f}')
    print(f'재현율: {recall:.4f}')
    print(f'F1 스코어: {f1:.4f}')
    print(f'ROC AUC: {roc_auc:.4f}')
    print('')

In [ ]:
# 원본 데이터를 재로딩 하고, feature데이터 셋과 Label 데이터 셋 추출.

y_titanic_df = titanic_df['Survived']
X_titanic_df= titanic_df.drop('Survived',axis=1)

X_titanic_df = transform_features(X_titanic_df)

In [ ]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test=train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=11)
X_train, X_test, y_train, y_test=train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=11, stratify=y_titanic_df)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 결정트리, Random Forest, 로지스틱 회귀를 위한 사이킷런 Classifier 클래스 생성
dt_clf = DecisionTreeClassifier(random_state=10)
rf_clf = RandomForestClassifier(random_state=10)
lr_clf = LogisticRegression(max_iter=2000, random_state=10)
print('dt_clf 학습')
print('='*12)
train_and_evaluate(dt_clf, X_train, X_test, y_train, y_test)
print('rf_clf 학습')
print('='*12)
train_and_evaluate(rf_clf, X_train, X_test, y_train, y_test)
print('lr_clf 학습')
print('='*12)
train_and_evaluate(lr_clf, X_train, X_test, y_train, y_test)

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'max_depth':[2,3,5,10,12],
             'min_samples_split':[2,3,5], 'min_samples_leaf':[1,5,8,10]}

grid_dclf = GridSearchCV(dt_clf , param_grid=parameters , scoring='accuracy' , cv=5)
grid_dclf.fit(X_train , y_train)

print('GridSearchCV 최적 하이퍼 파라미터 :',grid_dclf.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_dclf.best_score_))
best_dclf = grid_dclf.best_estimator_

train_and_evaluate(best_dclf, X_train, X_test, y_train, y_test)


In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'n_estimators':[10,100,200], 'max_depth':[2,3,5,10,12],
             'min_samples_split':[2,3,5], 'min_samples_leaf':[1,5,8,10]}

grid_rfclf = GridSearchCV(rf_clf , param_grid=parameters , scoring='accuracy' , cv=5)
grid_rfclf.fit(X_train , y_train)


print('GridSearchCV 최적 하이퍼 파라미터 :',grid_rfclf.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_rfclf.best_score_))
best_rfclf = grid_rfclf.best_estimator_

train_and_evaluate(best_rfclf, X_train, X_test, y_train, y_test)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10, 50, 100]
}
grid_lrclf = GridSearchCV(lr_clf , param_grid=param_grid , cv=5, verbose=0)
grid_lrclf.fit(X_train , y_train)

print('GridSearchCV 최적 하이퍼 파라미터 :',grid_lrclf.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_lrclf.best_score_))
best_lrclf = grid_lrclf.best_estimator_

train_and_evaluate(best_lrclf, X_train, X_test, y_train, y_test)